## 왜 TF-slim인가


TF-SLIM은 쉽게 인공신경망을 만들고 훈련시키고 평가하게 해주는 라이브러리이다.

- 표준적인 코드를 안쓰게 함으로써 훨씬 더 작게 모델을 정의하게 해준다. 이것은 argument scoping과 수 많은 high level layers와 variables를 사용함으로 가능하게 된다. 이런 툴들은 가독성과 유지보수성을 증가시키고 hyperparameter를 복사 붙혀넣기에서 나오는 에러를 줄이고 hyperparameter 튜닝을 간단하게 한다.

- 자주 사용되는 regularizer를 제공함으로써 모델을 만드는걸 간단하게 한다

- 많이 쓰이는 몇몇 컴퓨터 비전 모델이 slim으로 만들어져 있어서 사용가능하다. 이것들은 블랙박스로써 사용가능하고 내부 레이어에 여러 개의 앞부분을 추가함으러써 다양하게 확장가능하다

- slim은 복잡한 모델을 확장하기 쉽게 해주고 이미 존재하는 모델의 체크포인트를 사용함으로써 훈련 알고리즘을 미리 돌아진 상태로 만든다.!

## TF-slim의 다양한 요소엔 어떤게 있는가?

TF-slim은 독립적으로 존재하도록 만들어진 몇 가지 부분으로 구성된다. 다음과 같은 주요 부분으로 구성된다.

- arg_scope : arg_scope라고 불리는 새로운 scope를 제공하여 scope내에 있는 특정한 operation에 대한 default argument를 정의하도록 해준다.

- data : TF-slim의 dataset 정의, data providers, paraller_reader 그리고 decoding 유틸리티를 포함한다

- evaluateion : 모델을 평가하는데 흔히 사용되는 함수를 포함한다.

- layers : 텐서플로우를 사용해 모델을 만드는 high level 레이어를 포함한다

- learning : 모델을 훈련하는데 흔히 쓰이는 함수를 포함한다.

- losses : 주로 사용되는 loss 함수를 포함한다

- metrics : 자주 쓰는 평가 metriccs을 포함한다.

- nets : VGG나 AlexNet과 같은 유명한 네트워크 정의를 포함한다.

- queues : 쉽게 안전하게 QueueRunners를 시작하고 닫게 하는 context manager를 포함한다.

- regularizer : weight regularizer를 포함한다

- variables : 변수를 생성하고 조작을 하는 편리한 wrapper를 포함한다.

## 모델 정의하기

모델들은 변수와 레이어, 스코프를 조합함으로써 충분히 정의할 수 있다. 

### 변수들

기본 텐서플로우에서 변수를 만드는 것은 사전에 정의된 값이나 초기화 매커니즘을 요구한다. 나아가 만약 한 변수가 GPU같은 특정한 장치에서 만들어지려면 반드시 외부적으로 명시되어야 했다. 변수 생성을 위하 코드 요구사항을 완화하기 위해서 TF-Slim은 간소화된 wrapper 함수들을 제공하는데 caller가 쉽게 변수를 정의하도록 해준다.


예를 들어 weight 변수를 만들고 truncated normal로 초기화 하고 l2_loss로 regularize한 뒤에 CPU에 올라가기 위해선 다음과 같이 선언하면 된다.

```python
weights = slim.variable('weights', 
                        shape=[10,10,3,3],
                        initializer=tf.truncated_normal_initializer(stddev=0.1),
                        regularizer=slim.l2_regularizer(0.05),
                        device='/CPU:0')
```

기본 텐서플로우에선 2종류의 변수가 있다 : regular variables/ local(trasient) variables. 

대부분 변수는 regular variables이다 : 일단 만들어지면 saver를 사용해서 디스크에 저장할 수 있다. local variables은 세션이 돌아가는 중에만 존재하고 디스크에는 저장되지 않는다.

TF-Slim은 모델의 파라미터는 나타내는 **model variable**을 정의함으로써 변수들을 구분한다. Model variables은 학습 중에 학습되거나 fine-tuned되고 evaluation이나 inference 중에는 체크포인트를 불러올 수 있다. slim.fully_connected나 slim.conv2d레이어로 만들어지는 변수가 그 예들이다. Non-model variables들은 학습이나 평가과정에선 사용되지만 inference에서는 필요없는 변수들을 말한다. 예를 들어 global_step은 학습과 evaluation 중에는 사용되지만 실제론 모델의 일부가 아니다. 마찬가지로 moving average variables은 model variable처럼 만들 순 있지만 그 자체로는 model variables이 아니다.

model variables과 regular variables 둘 다 TF-Slim으로 만들고 불러올 수 있다.

```python
# Model Variables
weights = slim.model_variable('weights', 
                              shape=[10,10,3,3],
                              intializer=tf.truncated_normal_initializer(stddev=0.1),
                              regularizer=slim.l2_regularizer(0.05),
                              device='/CPU:0')

model_variables = slim.get_model_variables()

# Regular variables
my_var = slim.variable('my_var',
                       shape=[20, 1],
                       initializer=tf.zeros_intializer())

regular_variables_and_model_variables = slim.get_variables()
```

어떻게 이게 동작할까? TF-Slim의 레이어나 slim.model_variable 함수를 통해 model variable을 만들 때, TF-Slim은 변수를 tf.GraphKeys.MODEL_VARIABLES collection에 더한다. 만약 자신만의 레이어나 변수 생성 루틴이 있는 상태에서 TF-Slim이 model variable을 관리하게끔 하고 싶다면 어떨까? TF-Slim은 그러한 model variable을 collection에 더하는 편리한 함수를 제공한다.


```python
my_model_variable = CreateViaCustomCode()

# Letting TF-Slim know about the additional variable
slim.add_model_variable(my_model_varaible)

```

### 레이어

텐서플로우 operations은 꽤나 광범위하지만 인공신경망 개발자들은 보통 모델을 layer, losses, metrics 그리고 network같은 high-level 컨셉으로 생각한다. Convolutional layer, Fully Connected layer, BatchNorm layer같은 레이어는 단 하나의 텐서플로우 operation보다 더 압축적이고 여러개의 operation을 동반한다. 나아가서, 하나의 레이어는 보통(항상 그런 것은 아니지만) variable(tunable parameters)을 가지고 있는데 이는 primitive operations와는 차이가 나는 부분이다. 예를 들어 인공 신경망의 Convolutional layer는 몇몇 low-level operations으로 구성된다.

1. weight와 bias variable을 만든다.
2. 이전 레이어릐 인풋과 weight를 convolve시킨다
3. convolution 결과에 biase를 더한다
4. activation function을 적용한다.

일반적인 텐서플로우 코드를 사용하면 제법 노동이 따른다.

```python
input = ...
with tf.namme_scope('conv1_1') as scope:
    kernel = tf.Variable(tf.truncated_normal([3,3,64,128], dtype=tf.flaot32, stddev=1e-1), name='weight')
    conv = tf.nn.conv2d(input, kernel, [1,1,1,1], padding='SAME')
    biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32))
    bias = tf.nn.bias_add(conv, biases)
    conv1 = tf.nn.relue(bias, name=scope)
                        
```

이런 반복되는 코드를 줄이기 위해, TF-Slim은 인공 신경망 layer를 좀 더 함축시킨 수준으로 정의한 편리한 operations을 제공한다. 예를 들어 위의 코드와 이에 대응되는 TF-Slim 코드를 비교해보라.

```python
input = ...
net = slim.conv2d(input, 128, [3,3], scope='conv1_1')
```

TF-Slin은 인공신경망을 만들기 위해 필요한 다양한 구성요소에 대해 표준으로 구현된 코드를 제공한다.

- slim.bias_add
- slim.batch_normm
- slim.conv2d
- slim.conv2d_in_plane
- slim.conv2d_transpose
- slim.fully_connected
- slim.avg_pool2d
- slim.dropout
- slim.flatten
- slim.max_pool2d
- slim.one_hot_encoding
- slim.separable_conv2d
- slim.unit_norm



TF-Slim은 또한 repeat와 stack이라는 두 가지 meta-opeartion을 제공하는데 이는 똑같은 operation을 반복적으로 실행하게 해준다. 예를 들어 다음 pooling layer 사이에 연속해서 convolution을 수행해야 하는 VGG 네트워크의 코드 일부분을 보도록 해라.

```python
net = ...
net = slim.conv2d(net, 256, [3,3], scope='conv3_1')
net = slim.conv2d(net, 256, [3,3], scope='conv3_2')
net = slim.conv2d(net, 256, [3,3], scope='conv3_3')
net = slim.max_pool2d(net, [2,2], scope='pool2')
```

이런 반복적인 코드를 줄이는 한 가지 방법은 for loop을 사용하는 것이다.

```python
net = ...
for i in range(3):
    net = slim.conv2d(net, 256, [3,3], scope='conv3_%d' % (i+1))
    
net = slim.max_pool2d(net, [2,2], scope='pool2d')
```

TF-Slim의 repeat operation을 쓰면 더 깔끔하게 할 수 있다.

```python
net = slim.repeat(net, 3, slim.conv2d, 256, [3,3], socpe='conv3')
net = slim.max_pool2d(net, [2,2], socpe='pool2')
```

주막할 점은 slim.repeat는 같은 argument가 바로 적용될 뿐만 아니라 scope를 자동으로 풀도록 해주어서 연속해서 slim.conv2d를 호출할 때 밑줄 표시와 해당되는 숫자로 추가 표시되게 해준다. 더 자세히말하면, 이 예제에서 scope는 conv3/conv3_1, conv3/conv3_2, conv3/conv3_3로 이름이 붙혀진다.


추가로 TF-Slim의 slim.stack operatior는 caller가 똑같은 operation을 다른 argument로 적용시켜서 stack of layer나 tower of layers를 만들 수 있게 한다. slim.stack은 또한 매번 새로운 operation이 만들어질 때 마다 새로운 tf.variable_scope를 만든다. 예를 들어 간단하게 MLP(Multi-Layer Perceptron)을 만드는 방법은 다음과 같다.

```python
# Verbose way:
x = slim.fully_connected(x, 32, scope='fc/fc_1')
x = slim.fully_connected(x, 64, scope='fc/fc_2')
x = slim.fully_connected(x, 128, scope='fc/fc_3')

# Equivalent, TF-Slim way using slim.stack
slim.stack(x, slim.fully_connected, [32, 64, 128], scope='fc')
```

이번 예제에서 slim.stack은 slim.fully_connected를 3번 call 하는데 각 함수가 호출되어서 나오는 결과물을 다음 함수로 넘겨주면서 동작한다. 하지만 호출될때마다 hidden unit의 수는 32,64,128로 변한다. 마찬가지로 stack을 써서 여러개의 convolution의 tower를 만들 수 있다.

```python
# Verbose way
x = slim.conv2d(x, 32, [3,3], scope='core/core_1')
x = slim.conv2d(x, 32, [1,1], scope='core/core_2')
x = slim.conv2d(x, 64, [3,3], scope='core/core_3')
x = slim.conv2d(x, 64, [1,1], scope='core/core_4')

# Using stack:
slim.stack(x, slim.conv2d, [(32, [3,3]), (32, [1,1]), (64, [3,3]), (64, [1,1])], scope='core')
```


### 스코프

텐서플로우의 스코프 매처커니즘 형태에 추가하여, TF-Slim은 arg_scope라는 새로운 scoping 매커지즘을 추가한다. 이 새로운 scope는 하나 이상의 operation과 arg_scope에 정의된 각각의 oepration에 넘겨지는 argument를 표현하도록 해준다. 이 기능은 예제를 통해 가장 잘 나타낼 수 있다. 다음 코드 snippet을 고려하도록 하자

```python
net = slim.conv2d(inputs, 64, [11, 11], 4, padding='SAME',
                  weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                  weights_regularizer=slim.l2_regularizer(0.0005), scope='conv1')
net = slim.conv2d(net, 128, [11, 11], padding='VALID',
                  weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                  weights_regularizer=slim.l2_regularizer(0.0005), scope='conv2')
net = slim.conv2d(net, 256, [11, 11], padding='SAME',
                  weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                  weights_regularizer=slim.l2_regularizer(0.0005), scope='conv3')
```

명확히 이 세개의 convolution layer는 같은 hyperparameter를 많이 공유한다. 둘은 같은 padding을 가지고 세개 모두는 같은 weight_intializer와 weight_reguliarizer를 가진다. 이 코드는 가독성이 떨어지고 걸러낼만한 반복되는 값들을 많이 포함한다. 한 가지 해결책은 variable을 사용해서 default value를 명시하는 것이다.

```python
padding = 'SAME'
initializer = tf.truncated_normal_intializer(stddev=0.01)
regularizer = slim.l2_regularizer(0.0005)
net = slim.conv2d(inputs, 64, [11, 11], 4,
                  padding=padding,
                  weights_initializer=initializer,
                  weights_regularizer=regularizer,
                  scope='conv1')
net = slim.conv2d(net, 128, [11, 11],
                  padding='VALID',
                  weights_initializer=initializer,
                  weights_regularizer=regularizer,
                  scope='conv2')
net = slim.conv2d(net, 256, [11, 11],
                  padding=padding,
                  weights_initializer=initializer,
                  weights_regularizer=regularizer,
                  scope='conv3')
```

이 방법은 모든 convolution이 정확히 똑같은 parameter 값을 공유하도록 해주지만 난잡한 코드를 깔끔하게 해주진 않는다. arg_scope를 사용함으로써 우리는 각 레이어가 같은 값을 쓰면서 코드를 간단히 할 수 있다.

```python
with slim.arg_scope([slim.conv2d], padding='SAME',
                    weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                    weights_regularizer=slim.l2_regularizer(0.0005)):
    net = slim.conv2d(inputs, 62, [11,11], scope='conv1')
    net = slim.conv2d(net, 128, [11, 11], padding='VALID', scope='conv2')
    net = slim.conv2d(net, 256, [11, 11], scope='conv3')
```

예제에서 보져주듯이 arg_scope를 사용하면 코드를 깔끔하게 하고 유지보수하기 쉽게 해준다. 주목할 점은 arg_scope안에 argument 값이 명시되더라도 지역변수로 덮어쓸 수 있다. 특히나 padding argument는 SAME으로 설정되었지만 두번째 convolution는 이 값을 VALID로 덮어씌웠다.

또한 argu_scope는 끼워 넣어 쓸 수 있는데 같은 scope안에 여러개의 operation을 쓸 수 있다.

```python
with slim.arg_scope([slim.conv2d, slim.fully_connected],
                    activation_fn=tf.nn.relu,
                    weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                    weights_regularizer=slim.l2_regularizer(0.0005)):
    with slim.arg_scope([slim.conv2d], stride=1, padding='SAME'):
        net = slim.conv2d(inputs, 64, [11, 11], 4, padding='VALID', scope='conv1')
        net = slim.conv2d(net, 256, [5,5],
                          weights_initilizer=tf.truncated_normal_initializer(stddev=0.03),
                          scope='conv2')
        net = slim.fully_connected(net, 1000, activation_fn=None, scope='fc')
          
```

이 예제에서 첫번째 arg_scope는 같은 weights_intializer와 weights_regularizer를 scope내에 있는 conv2d와 fully_connected에 적용한다. 두번째 arg_socpe에선 추가적인 default argument를 conv2d에만 명시하였다.

### 실제 예제 : VGG16 레이어 만들기

TF-Slim의 Variables, Operation, scope를 합쳐서 우리는 보통은 매우 복잡한 네트워크를 단 몇줄만으로 표현할 수 있다. 예를 들어 전체 VGG 구조를 다음의 snippet으로 정의할 수 있다.

```python
def vgg16(inputs):
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                        activation_fn=tf.nn.relu,
                        wegihts_initializer=tf.truncated_normal_initializer(0.0, 0.01),
                        weights_regularizer=slim.l2_regularizer(0.0005)):
        net = slim.repeat(inputs, 2, slim.conv2d, 64, [3,3], scope='conv1')
        net = slim.max_pool2d(net, [2,2], scope='pool1')
        net = slim.repeat(net, 2, slim.conv2d, 128, [3,3], scope='conv2')
        net = slim.max_pool2d(net, [2,2], scope='pool2')
        net = slim.repeat(net, 3, slim.conv2d, 256, [3,3], scope='conv3')
        net = slim.max_pool2d(net, [2,2], scope='pool3')
        net = slim.repeat(net, 3, slim.conv2d, 512, [3,3], scope='conv4')
        net = slim.max_pool2d(net, [2,2], scope='pool4')
        net = slim.repeat(net, 3, slim.conv2d, 512, [3,3], scope='conv5')
        net = slim.max_pool2d(net, [2,2], scope='pool5')
        net = slim.fully_connected(net, 4096, scopoe='fc6')
        net = slim.dropout(net, 0.5, scope='dropout6')
        net = slim.fully_connected(net, 4096, scope='fc7')
        net = slim.dropout(net, 0.5, scope='dropout7')
        net = slim.fully_connected(net, 1000, activation_fn=None, scope='fc8')
        
  return net         
```



### 모델 학습하기

텐서플로우 모델을 학습하는 것은 모델, loss function, gradient computation 그리고 순회하면서 loss에 대한 model의 weight에 대한 gradient를 계산하고 이에 맞춰서 weight를 업데이트하는 학습 과정을 요구한다. TF-Slim은 자주 쓰는 loss 함수와 학습과 평가 과정에 돌아가는 helper 함수를 제공한다.

#### Losses

loss 함수는 우리가 최소화하고자 하는 양을 정의한다. classifaction 문제에선 보통 class들에 대한 true distribution과 예측되는 확률 분포 사이의 cross entropy를 사용한다. regression 문제에선 종종 예측값과 true 값간의 차이를 sum-of-squars를 구해서 사용한다.

multi-task learning model과 같은 특정 모델은 여러개의 loss 함수를 사용해야 한다. 다시 말해서, 궁극적으로 최소화되는 loss 함수는 다른 loss 함수들의 합이다. 예를 들어 한 모델이 이미지의 장면 종류와 픽셀로 부터 얻은 이미지에서 깊이를 예측한다고 하자. 이 모델의 loss 함수는 classification loss와 depth 예측 loss의 합일 것이다. 

TF-Slim은 losses 모듈을 통해 loss함수를 정의하고 기록할 수 있는 다루기 쉬운 매커니즘을 제공한다. VGG 네트워크를 학습하고자 하는 다음 예제를 보도록 하자.

```python
import tensorflow as tf
import tesnorflow.contrib.slim.nets as nets

vgg = nets.vgg

# Load the images and labels
images, labels =  ...

# Create the model
predictions, _ = vgg.vgg_16(images)

# Define the loss functions and get the total loss.
loss = slim.losses.softmax_cross_entropy(predictions, labels)

```

이 예제에서 우리는 모델을 만들면서 시작하는데 일반적인 classifcation loss를 더한다. 이제 multiple output을 만드는 multi-task model 케이스로 돌아가보자.

```python
# Load the images and lables
images, scene_labels, depth_labels = ...

# Create the model
scene_predictions, depth_predictions = CreateMultiTaskModel(images)

# Define the loss functions and get the total loss.
classification_loss = slim.losses.softmax_cross_entropy(scene_predictions, scene_labels)
sum_of_squares_loss = slim.losses.sum_of_squares(depth_predictions, depth_labels)

# The following two lines have the same effect:
total_loss = classification_loss + sum_of_squares_loss

total_loss = slim.losses.get_total_loss(add_regularization_losses=False)

```

이 예제에서 우리는 slim.losses.softmax_cross_entropy와 slim.losses.sum_of_squares를 call해서 만든 2개의 loss를 지닌다. 이 둘을 더하거나 slim.losses.get_total_loss()를 부름으로써 전체 loss를 구할 수 있다. 어떻게 이게 동작할까? TF-Slim을 통해서 loss 함수를 만들때, TF-Slim은 loss함수에 대한 특별한 텐서플로우 collection에 loss를 더한다. 이것은 당신이 일일이 total loss를 관리하거나 TF-Slim이 대신해서 관리하게끔 해준다. 

만약 사용자가 정의한 loss 함수인데 TF-Slim에서 관리하게끔 하고 싶다면 어떨까? loss_ops.py는 해당 loss를 TF-Slim collection에 더하도록 하는 함수를 가지고 있다.

```python
# Load the images and labels
images, scene_labels, depth_labels, pose_labels = ...

# Create the model
scnene_predictions, depth_predictions, pose_predictions = CreateMultiTaskModel(images)

# Define the loss functions and get the total loss.
classification_loss = slim.losses.softmax_cross_entropy(scene_predictions, scene_labels)
sum_of_squares_loss = slim.losses.sum_of_squares(depth_predictions, depth_labels)
pose_loss = MyCustomLossFunction(pose_predictions, pose_labels)
slim.loses.add_loss(pose_lose) # Letting TF-Slim know about the additional loss.

# The following two ways to compute the total loss are equivalent:
regularization_loss = tf.add_n(slim.losses.get_regularization_losses())
total_loss1 = classification_loss + sum_of_squares_loss + pose_loss + regularization_loss

# (Regularization loss is included in the total loss by default).
total_loss2 = slim.losses.get_total_loss()
```

이 예제에서 우리는 수동으로 total loss 함수를 만들거나 TF-Slim이 추가적인 loss에 대해 알게하여 이 loss를 알아서 처리하도록 할 수 있다.

### 학습 루프

TF-Slim은 모델을 학습시키기 위한 간단하지만 강력한 툴을 제공하는데 learning.py에 위치한다. 이러한 것들에는 반복적으로 loss를 계산하는 함수, gradient를 계산하는 함수, 모델을 디스크에 저장하는 함수, gradient를 쉽게 조작하게 해주는 몇몇 함수들이 있다. 예를 들어 일단 우리가 모델, loss 함수와 optimization scheme을 구축한다음에는 우리는 slim.learning.create_train_op와 slim.learning.train을 호출하여 optimization을 돌릴 수 있다.

```python
g = tf.Graph()

# Create the model and specify the losses...
...

total_loss = slim.losses.get_total_loss()
optimizer = tf.train.GradientDescentOptimizer(learning_rate)

# create_train_op ensures that each time we ask fo the loss, update_ops
# are run and the gradients being computed are applied too
train_op = slim.learning.create_train_op(total_loss, optimizer)
logdir = ... # Where checkpoints are stored.

slim.learning.train(train_op, logdir, number_of_steps=1000, save_summaries_secs=300, save_interval_secs=600)
```

이 예제에서 train_op와 함께 slim.learning.train이 나오는데 loss를 계산하고 gradient step을 적용한다. logdir은 체크포인트와 이벤트 파일이 저장되는 디렉토리를 나타낸다. 우리는 gradient steps을 어떤 숫자로든지 제한할 수 있다. 이번 경우에 1000번 step이 적용되도록 설정하였다. 마지막으로 save_summaries_secs=300은 우리가 매번 5분마다 summaries를 계산하는 것을 나타내고 save_interval_secs=600은 매 10분마다 모델의 체크포인트를 저장하는 것을 나타낸다.

### 실제 예제 : VGG16 모델 학습하기

이 과정을 나타내기 위해 다음의 VGG 네트워크를 학습하는 샘플을 살펴보자.

```python
import tensorflow as tf
import tensorflow.contrib.slim.nets as nets

slim = tf.contrib.slim
vgg = nets.vgg

...

train_log_dir = ...
if not tf.gfile.Exists(train_log_dir):
    tf.gfile.MakeDirs(train_log_dir)
    
with tf.Graph().as_default():
    # Set up the data loading
    images, labels = ...
    
    # Define the model:
    predictions = vgg.vgg_16(images, is_training=True)
    
    # Specify the loss function:
    slim.losses.softmax_cross_entropy(predicitons, labels)
    
    total_loss = slim.losses.get_total_loss()
    tf.summary.scalar('losses/total_loss', total_loss)
    
    # Specify the optimization scheme
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=.001)
    
    # create_train_op that ensures that when we evaluate it to get the loss,
    # the update_ops are done and the gradient updates are computed.
    train_tensor = slim.learning.create_train_op(total_loss, optimizer)
    
    # Actually runs training
    slim.learning.train(train_tensor, train_log_dir)

```

## 존재하는 모델을 Fine-Tuning하기


### 체크포인트로부터 변수를 복구하는 방법

모델의 학습이 끝나면, tf.train.Saver()를 통해 복원할 수 있는데 주어진 체크포인트에서 변수들을 되돌려준다. 많은 경우에 tf.train.Saver()는 전체 혹은 일부 variables을 복구하는 간단한 매커니즘을 제공한다.

```python
# Create some variables
v1 = tf.Variable(..., name="v1")
v2 = tf.Variable(..., name="v2")

...
# Add ops to restore all the variables.
restorer = tf.train.Saver()

# Add ops to restore some variabels
restorer = tf.train.Saver([v1, v2])

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
    # Restore variables from disk.
    restorer.restore(sess, "/tmp/model.ckpt")
    print("Model restroed.")
    # Do some work with the model
    ...

```

### 부분적으로 모델 복구하기

종종 완전히 새로운 데이터셋이나 새로운 작업에 대해 사전에 학습된 모델을 fine-tune하는게 요구된다. 이런 상황에선 TF-Slim의 helper 함수들을 사용해서 일부 variable만 복원하도록 선택할 수 있다.

```python
v1 = slim.variable(name="v1",...)
v2 = slim.variable(name="nested/v2", ...)
...

# Get list of variables to restore (which contains only 'v2'). 
# These are all equivalent methods
variables_to_restore = slim.get_variables_by_name("v2")
# or
varaibles_to_restore = slim.get_varaibes_by_suffix("2")
# or
variables_to_restore = slim.get_variables(scope="nested")
# or 
variables_to_restore = slim.get_varaibles_to_restore(include=["nested"])
# or
variables_to_restore = slim.get_variables_to_restore(exclude=["v1"])

# Create the saver which will be used to restore the variables
restorer = tf.train.Saver(variables_to_restore)

with tf.Session() as sess:
    # Restore variables from disk.
    restorer.restore(sess, "/tmp/model.ckpt")
    print("Model restored.")
    
```

### 다른 variable 이름으로 모델 복원하기

한 체크포인트 에서 variables를 복원할때 Saver는 체크포인트 파일에서 varaible name을 찾고 그것들을 현재 그래프에 있는 variable에 매핑시킨다. 위에서 우리는 saver를 하나 만들고 거기에 variables 리스트를 넘겨주었다. 이런 경우에 체크포인트 파일에 위치한 variable name이 넘겨준 varaible의 var.op.name으로 부터 내부적으로 얻어진다. 

이런 방식은 체크포인트에 있는 variable name과 그래프에 있는 이름이 매칭이 될때 잘 동작한다. 하지만 때때로 체크포인트에 있는 variable name이 현재 그래프에 있는 variable과 다른 경우에 모델을 복원하기 원한다. 이런 경우엔 반드시 Saver에게 각각의 체크포인트 variable name과 각 그래프에 있는 variable에 매핑되는 딕셔너리를 제공해야 한다. 간단한 함수 하나로 체크포인트 variable name을 얻는 다음 예제를 보도록 하자.

```python
# Assuming that 'conv1/weights/ should be restroe from 'vgg16/conv1/weights/
def name_in_checkpoint(var):
    return 'vgg16/'+var.op.name

# Assuming that 'conv1/weigts' and 'conv1/bias' should be restored from 'conv1/params1' and 'conv1/params2'

def name_in_checkpoint(var):
    if "weights" in var.op.name:
        return var.op.name.replace("weights", "params1")
    if "bias" in var.op.name:
        return var.op.name.repace("bias", "params2")
    
    
variables_to_restore = slim.get_model_variables()
variables_to_restore = {name_in_checkpoint(var): var for var in variables_to_restore}
restorer = tf.train.Saver(variables_to_restore)

with tf.Session() as sess:
    # Restore variables from disk.
    restorer.restore(sess, "/tmp/model.ckpt")
```


### 다른 작업에 대해 모델 Fine-Tuning하기

이미 학습된 VGG16 모델이 있다고 해보자. 이 모델은 1000개의 class로 구성된 이미지넷 데이터셋에 대해 학습된 상태이다. 하지만 우리는 20개의 class만 가지고 있는 Pascal VOC 데이터셋에 이 모델을 적용하려고 할 수 있다. 이렇게 하려면 우리는 마지막 레이어를 제외한 학습된 모델의 값을 사용해서 새로운 모델을 초기화할 수 있다.

```python
# Load the Pascal VOC data
image, label = MyPascalVocDataLoader(...)
images, labels = tf.train.batch([image, label], batch_size=32)

# Create the model
predictions = vgg.vgg_16(images)

train_op = slim.learning.create_train_op(...)

# Specify where the Model, trained on ImageNet, was saved.
model_path = '/path/to/pre_trained_on_imagenet.checkpoint'

# Speicify where the new model will live:
log_dir = '/path/to/my_pascal_mdoel_dir/'

# Restore only the convolutional layers:
variables_to_restore = slim.get_variables_to_restore(exclude=['fc6', 'fc7', 'fc8'])
init_fn = assign_from_checkpoint_fn(model_path, variables_to_restore)

# Start training
slim.learning.train(train_op, log_dir, init_fn=init_fn)
```

## 모델 평가하기

일단 모델을 학습되면(혹은 한참 학습중일때) 우리는 실제로 모델의 성능이 얼마나 뛰어난지 보고싶어 한다. 이것은 모델의 성능을 평가하는 **evaluation metrics**과 데이터를 올리고 inference를 하고 결과를 ground truth에 비교하여 평가 점수를 매기는 **evaluation code**를 선택함으로써 할 수 있다. 이 단계는 단 한번만 실행될 수도 있고 주기적으로 반복될 수도 있다


### Metrics

우리는 성능평가치로 사용될(loss 함수가 아니다. loss는 훈련 중에 바로 최적화된다) metric을 하나 정의하지만 모델을 평가하기 위한 목적에 관심이 있다. 예를 들어 log loss를 줄이려고 할 수 있지만 사용할 metrics은 F1 score(test accuracy)일 수도 있고 IOU(Intersection Over Union) score 일수도 있다. IOU score는 미분 불가능하기 때문에 losses로는 사용될 수 없다.

TF-Slim은 여러 metric operation을 제공하는데 모델을 쉽게 평가하도록 해준다. 개략적으로 metric 값을 계산하는 것은 3가지로 나눌 수 잇다.

1. Initilaization : metric을 계산하는데 사용되는 variables을 초기화한다.
2. Aggergation : metric을 계산하는데 사용되는 operation(sums, etc)를 수행한다.
3. Finalization : 부가적으로 metric 값을 계산하기 위한 최종적인 operation을 수행한다. 예를 들어 평균, 최소값, 최대값 등을 계산하는 것을 말한다.

예를 들어, meamn_absolute_error를 계산하려면 두 개의 variables(count and total)이 0으로 초기화된다. aggregation 중에는 우리는 예측값과 label값을 봐서 차이값을 계산하고 total에 더하도록 한다. 매번 다른 값이 나올때 마다, count가 증가된다. 마지막으로 finalization 중에는 total을 count로 나눠서 mean을 얻는다.

다음 예제는 metric을 선언하는 API을 보여준다. metrics은 종종 학습 데이터셋(loss가 계산되는)과 다른 test 데이터셋으로 평가되기 때문에 test 데이터를 사용한다고 가정한다. 

```python
images, labels = LaodTestData(...)
predictions = MyModel(images)

mae_value_op, mae_update_op = slim.metrics.streaming_mean_absolute_error(predictions, labels)
mre_value_op, mre_update_op = slim.metrics.streaming_mean_relative_error(predictions, labels)
pl_value_op, pl_update_op = slim.metrics.precentage_less(mean_relative_errors, 0.3)
```

예제가 보여주듯이, metric을 하나 만들면 2개의 값이 리턴된다: value_op, update_op. value_op는 멱등 operation으로 metric의 현재 값을 리턴해준다. update_op는 위에서 언급된 aggregation 단계를 수행하고 metric 값을 리턴하는 operation이다.

각기 다른 value_op와 update_op를 기록하는 것은 노동이 될 수 있다. 이것을 처리하기 위해 TF-Slim은 2가지 편리한 함수를 제공한다.

```python
# Aggregates the value and update ops in two lists:
value_ops, update_ops = slim.metrics.aggregate_metrics(
    slim.metrics.streaming_mean_absolute_error(predictions, labels),
    slim.metrics.streaming_mean_squared_error(predicitons, labels))

# Aggregates the value and update ops in two dictionaries:
names_to_values, names_to_updates = slim.metrics.aggregate_metric_map({
    "eval/mean_absolute_error": slim.metrics.streaming_mean_absolute_error(predictions, labels),
    "eval/mean_squared_error": slim.metrics.streaming_mean_squared_error(predictions, labels),
})
```

### 실제 예제 : 여러 metrics을 기록하기

```python
import tensorflow as tf
import tensorflow.contrib.slim.nets as nets

slim = tf.contrib.slim
vgg = nets.vgg

# Load the data
images, labels = load_data(...)

# Define the network
predictions = vgg.vgg_16(images)

# Choose the metrics to compute
names_to_values, names_to_updates = slim.metrics.aggregate_metrics_map({
    "eval/mean_absolute_error": slim.metrics.streaming_mean_absolute_error(predictions, labels),
    "eval/mean_squared_error": slim.metrics.streaming_mean_squared_error(predictions, labels),
})

# Evaluate the model using 1000 batches of datga:
num_batches = 1000

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    for batch_id in range(num_batches):
        sess.run(names_to_updates.values())
        
    metric_values = sess.run(names_to_values.values())
    for metric, value in zip(names_to_values.keys(), metric_values):
            print('Metric %s has value: %f' % (metric, value))
            
```

metric_op.py는 layers.py나 loss_ops.py 없이도 단독으로 사용될 수 있다는 것에 주목하자.


### 평가 loop

TF-Slim은 evaluation 모듈(evaluation.py)를 제공하는데 metric_ops.py 모듈에 있는 metrics들을 사용한 모델 평가 스크립트를 작성하게 해주는 helper 함수들이 들어있다. 이런 함수들에는 주기적으로 평가하는 함수, 배치 데이터에 대해 metrics을 평가하는 함수 그리고 metric 결과를 요약하는 함수들이 있다. 

```python
import tensorflow as tf

slim = tf.contrib.slim

# Load the data
images, labels = load_data(...)

# Define the network
predictions = MyModel(images)

# Choose the metrics to compute:
names_to_values, names_to_updates = slim.metrics.aggregate_metric_map({
    'accuarcy': slim.metrics.accuracy(predictions, labels),
    'precision': slim.metrics.precision(predictions, labels),
    'recall' : slim.metrics.recall(mean_relative_errors, 0.3),
})

# Create the summary ops such that they also print out to std output:
summary_ops = []
for metric_name, metric_value in names_to_values.iteritems():
    op = tf.summary.scalar(metric_name, metric_value)
    op = tf.Print(op, [metric_value], metric_name)
    summary_ops.append(op)
    
num_examples = 10000
batch_size = 32
num_batches = math.ceil(num_examples / float(batch_size))

# Setup the global step
slim.get_or_create_global_step()

output_dir = ... # Where the summaries are stored
eval_interval_secs = ... # How often to run the evaluation
slim.evaluation.evaluation_loop(
    'local',
    checkpoint_dir,
    log_dir,
    num_evals=num_batches,
    eval_op=names_to_updates.values(),
    summary_op=tf.summary.merge(summary_ops),
    eval_interval_secs=eval_interval_secs)

```

# TensorFlow - Slim image classification model library

TF-slim은 새로운 가벼운 high-level APU로써 복잡한 모델을 정의하고 학습하고 평가할 수 있다. 이 디렉토리는 TF-slim을 사용하여 

## TF-Slim Dataset Descriptor 만들기

일단 TFRecord 파일이 만들어지면, Slim Dataset을 간단히 정의할 수 있는데 데이터 파일에 대한 포인터를 저장할 뿐만 아니라 class labels, train/test split, 어떻게 TFExample proto를 파싱할지에 대한 metadata도 담고 있다. 우리는 Cifar10, ImageNet, Flowers와 MNIST에 대한 TF-Slim Dataset descriptor를 포함시켰다. TF-Slim DatasetDataProvider를 사용해서 TF-Slim dataset descriptor를 사용해서 데이터를 로딩하는 방법은 다음과 같다.

```python
import tensorflow as tf
from datasets import flower

slim = tf.contrib.slim

# Select the 'validation' dataset
dataset = flowers.get_split('validation', DATA_DIR)

# Creates a TF-Slim DataProvider which reads the dataset in the background
# during both training and testing
provider = slim.dataset_data_provider.DatasetDataProvider(dataset)
[image, label] = provider.get(['image', 'label'])
                            
```